# operator_examples.ipynb
# WESmith 06/29/22

In [1]:
import operator_utils as ws
import sympy as sy
import numpy as np
import matplotlib.pyplot as plt
from   sympy import sin, cos, latex
from   IPython.display import display, Math

In [2]:
def Arfken_example_2_4_2(ops):
    # ops is an Operators object defined with standard spherical coords r, theta, phi
    # this was successfully validated against Arfken result p. 83 6/18/22
    r, theta, phi = ops.coords_p
    Aphi = sy.symbols('A_phi', cls=sy.Function, real=True)
    Aphi = Aphi(r, theta)
    A    = sy.Matrix([0, 0, Aphi])
    v1   = ops.curl(ops.curl(A))
    tmp  = []
    for j, k in zip(v1, A):
        tmp.append(j + ops.Laplacian_scalar(k) - ops.Laplacian_symbolic(k))
    reduced = sy.trigsimp(ops.Zp.T * sy.Matrix(tmp))
    full    = sy.trigsimp(ops.Zp.T * v1)
    return full, reduced

# 3D spherical coordinates

In [3]:
# specifying positive and real variables helps in simplification of square-root expressions
x, y, z          = sy.symbols('x y z', real=True)
r, theta, phi    = sy.symbols('r theta, phi', positive=True, real=True)
orig_coords      = (x, y, z)
primed_coords    = (r, theta, phi)
orig_from_primed = (r * sin(theta) * cos(phi), 
                    r * sin(theta) * sin(phi), 
                    r * cos(theta))

In [4]:
dd = ws.Operators(orig_coords, primed_coords, orig_from_primed)

In [5]:
full, reduced = Arfken_example_2_4_2(dd)
full

Matrix([[\hat{\mathbf{e_\phi}}*(-Derivative(A_phi(r, theta), (r, 2)) - 2*Derivative(A_phi(r, theta), r)/r + A_phi(r, theta)/(r**2*sin(theta)**2) - Derivative(A_phi(r, theta), (theta, 2))/r**2 - Derivative(A_phi(r, theta), theta)/(r**2*tan(theta)))]])

In [6]:
reduced

Matrix([[\hat{\mathbf{e_\phi}}*(-\nabla^{2}(A_phi(r, theta)) + A_phi(r, theta)/(r**2*sin(theta)**2))]])

In [7]:
dd.metric

Matrix([
[1,    0,                  0],
[0, r**2,                  0],
[0,    0, r**2*sin(theta)**2]])

In [8]:
dd.metric_sqrt

Matrix([
[1, 0,            0],
[0, r,            0],
[0, 0, r*sin(theta)]])

In [9]:
dd.metric_inv_sqrt, dd.det, dd.det_sqrt

(Matrix([
 [1,   0,                0],
 [0, 1/r,                0],
 [0,   0, 1/(r*sin(theta))]]),
 r**4*sin(theta)**2,
 r**2*sin(theta))

In [10]:
dd.gradient()

Matrix([
[                 Derivative(F(r, theta, phi), r)],
[           Derivative(F(r, theta, phi), theta)/r],
[Derivative(F(r, theta, phi), phi)/(r*sin(theta))]])

In [11]:
dd.divergence()

Derivative(V_r(r, theta, phi), r) + V_\theta(r, theta, phi)*cos(theta)/(r*sin(theta)) + 2*V_r(r, theta, phi)/r + Derivative(V_\theta(r, theta, phi), theta)/r + Derivative(V_\phi(r, theta, phi), phi)/(r*sin(theta))

In [12]:
dd.curl()

Matrix([
[V_\phi(r, theta, phi)*cos(theta)/(r*sin(theta)) + Derivative(V_\phi(r, theta, phi), theta)/r - Derivative(V_\theta(r, theta, phi), phi)/(r*sin(theta))],
[                                  -Derivative(V_\phi(r, theta, phi), r) - V_\phi(r, theta, phi)/r + Derivative(V_r(r, theta, phi), phi)/(r*sin(theta))],
[                                          Derivative(V_\theta(r, theta, phi), r) + V_\theta(r, theta, phi)/r - Derivative(V_r(r, theta, phi), theta)/r]])

In [13]:
dd.curl(vector=False)

Matrix([[\hat{\mathbf{e_\phi}}*(Derivative(V_\theta(r, theta, phi), r) + V_\theta(r, theta, phi)/r - Derivative(V_r(r, theta, phi), theta)/r) + \hat{\mathbf{e_\theta}}*(-Derivative(V_\phi(r, theta, phi), r) - V_\phi(r, theta, phi)/r + Derivative(V_r(r, theta, phi), phi)/(r*sin(theta))) + \hat{\mathbf{e_r}}*(V_\phi(r, theta, phi)*cos(theta)/(r*sin(theta)) + Derivative(V_\phi(r, theta, phi), theta)/r - Derivative(V_\theta(r, theta, phi), phi)/(r*sin(theta)))]])

In [14]:
dd.Laplacian_scalar() # internally it takes divergence(gradient())

Derivative(F(r, theta, phi), (r, 2)) + 2*Derivative(F(r, theta, phi), r)/r + Derivative(F(r, theta, phi), (theta, 2))/r**2 + Derivative(F(r, theta, phi), theta)/(r**2*tan(theta)) + Derivative(F(r, theta, phi), (phi, 2))/(r**2*sin(theta)**2)

In [15]:
vv = dd.Laplacian_vector_full()

In [16]:
# successfully validated against Arfken p.82
vr = dd.Laplacian_vector_reduced()

In [17]:
vr[0]

\nabla^{2}(V_r(r, theta, phi)) - 2*V_\theta(r, theta, phi)/(r**2*tan(theta)) - 2*V_r(r, theta, phi)/r**2 - 2*Derivative(V_\theta(r, theta, phi), theta)/r**2 - 2*Derivative(V_\phi(r, theta, phi), phi)/(r**2*sin(theta))

In [18]:
vr[1]

\nabla^{2}(V_\theta(r, theta, phi)) - V_\theta(r, theta, phi)/(r**2*sin(theta)**2) + 2*Derivative(V_r(r, theta, phi), theta)/r**2 - 2*cos(theta)*Derivative(V_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2)

In [19]:
vr[2]

\nabla^{2}(V_\phi(r, theta, phi)) - V_\phi(r, theta, phi)/(r**2*sin(theta)**2) + 2*Derivative(V_r(r, theta, phi), phi)/(r**2*sin(theta)) + 2*cos(theta)*Derivative(V_\theta(r, theta, phi), phi)/(r**2*sin(theta)**2)

In [20]:
# test curl(grad): it should be 0
dd.curl(dd.gradient())

Matrix([
[0],
[0],
[0]])

In [21]:
# test div(curl): it should be 0
ws.pp(dd.divergence(dd.curl()))

<IPython.core.display.Math object>

In [22]:
dd.curl(dd.curl())

Matrix([
[Derivative(V_\theta(r, theta, phi), r, theta)/r + cos(theta)*Derivative(V_\theta(r, theta, phi), r)/(r*sin(theta)) + Derivative(V_\phi(r, theta, phi), phi, r)/(r*sin(theta)) + V_\theta(r, theta, phi)*cos(theta)/(r**2*sin(theta)) + Derivative(V_\theta(r, theta, phi), theta)/r**2 - Derivative(V_r(r, theta, phi), (theta, 2))/r**2 - cos(theta)*Derivative(V_r(r, theta, phi), theta)/(r**2*sin(theta)) + Derivative(V_\phi(r, theta, phi), phi)/(r**2*sin(theta)) - Derivative(V_r(r, theta, phi), (phi, 2))/(r**2*sin(theta)**2)],
[                                                                                                                                                                                  -Derivative(V_\theta(r, theta, phi), (r, 2)) - 2*Derivative(V_\theta(r, theta, phi), r)/r + Derivative(V_r(r, theta, phi), r, theta)/r + Derivative(V_\phi(r, theta, phi), phi, theta)/(r**2*sin(theta)) + cos(theta)*Derivative(V_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2) - Derivative(V

In [23]:
dd.gradient(dd.divergence())

Matrix([
[   Derivative(V_r(r, theta, phi), (r, 2)) + 2*Derivative(V_r(r, theta, phi), r)/r + Derivative(V_\theta(r, theta, phi), r, theta)/r + cos(theta)*Derivative(V_\theta(r, theta, phi), r)/(r*sin(theta)) + Derivative(V_\phi(r, theta, phi), phi, r)/(r*sin(theta)) - V_\theta(r, theta, phi)*cos(theta)/(r**2*sin(theta)) - 2*V_r(r, theta, phi)/r**2 - Derivative(V_\theta(r, theta, phi), theta)/r**2 - Derivative(V_\phi(r, theta, phi), phi)/(r**2*sin(theta))],
[Derivative(V_r(r, theta, phi), r, theta)/r - V_\theta(r, theta, phi)/r**2 - V_\theta(r, theta, phi)*cos(theta)**2/(r**2*sin(theta)**2) + Derivative(V_\theta(r, theta, phi), (theta, 2))/r**2 + 2*Derivative(V_r(r, theta, phi), theta)/r**2 + cos(theta)*Derivative(V_\theta(r, theta, phi), theta)/(r**2*sin(theta)) + Derivative(V_\phi(r, theta, phi), phi, theta)/(r**2*sin(theta)) - cos(theta)*Derivative(V_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2)],
[                                                                                    

In [29]:
dd.Z

Matrix([
[\hat{\mathbf{e_x}}],
[\hat{\mathbf{e_y}}],
[\hat{\mathbf{e_z}}]])

In [30]:
dd.Zp 

Matrix([
[     \hat{\mathbf{e_r}}],
[\hat{\mathbf{e_\theta}}],
[  \hat{\mathbf{e_\phi}}]])

In [31]:
dd.metric

Matrix([
[1,    0,                  0],
[0, r**2,                  0],
[0,    0, r**2*sin(theta)**2]])

In [32]:
dd.metric_sqrt

Matrix([
[1, 0,            0],
[0, r,            0],
[0, 0, r*sin(theta)]])

In [33]:
dd.metric_inv

Matrix([
[1,       0,                      0],
[0, r**(-2),                      0],
[0,       0, 1/(r**2*sin(theta)**2)]])

In [34]:
dd.metric_inv_sqrt

Matrix([
[1,   0,                0],
[0, 1/r,                0],
[0,   0, 1/(r*sin(theta))]])

In [35]:
dd.Jacobian

Matrix([
[sin(theta)*cos(phi), r*cos(phi)*cos(theta), -r*sin(phi)*sin(theta)],
[sin(phi)*sin(theta), r*sin(phi)*cos(theta),  r*sin(theta)*cos(phi)],
[         cos(theta),         -r*sin(theta),                      0]])

In [38]:
ws.pp(dd.Jacobian_inv)

<IPython.core.display.Math object>

In [40]:
ws.pp(dd.primed_basis)

<IPython.core.display.Math object>

# 3D cylindrical coordinates

In [41]:
x, y, z          = sy.symbols('x y z', real=True)
rho, phi         = sy.symbols('rho phi', positive=True, real=True)
orig_coords      = (x, y, z)
primed_coords    = (rho, phi, z)
orig_from_primed = (rho * cos(phi), rho * sin(phi), z)

In [42]:
aa = ws.Operators(orig_coords, primed_coords, orig_from_primed)

In [44]:
aa.metric

Matrix([
[1,      0, 0],
[0, rho**2, 0],
[0,      0, 1]])

In [45]:
aa.metric_sqrt

Matrix([
[1,   0, 0],
[0, rho, 0],
[0,   0, 1]])

In [46]:
aa.metric_inv

Matrix([
[1,         0, 0],
[0, rho**(-2), 0],
[0,         0, 1]])

In [47]:
aa.metric_inv_sqrt

Matrix([
[1,     0, 0],
[0, 1/rho, 0],
[0,     0, 1]])

In [48]:
aa.gradient()

Matrix([
[    Derivative(F(rho, phi, z), rho)],
[Derivative(F(rho, phi, z), phi)/rho],
[      Derivative(F(rho, phi, z), z)]])

In [49]:
aa.divergence()

Derivative(V_\rho(rho, phi, z), rho) + Derivative(V_z(rho, phi, z), z) + V_\rho(rho, phi, z)/rho + Derivative(V_\phi(rho, phi, z), phi)/rho

In [50]:
aa.curl()

Matrix([
[                              -Derivative(V_\phi(rho, phi, z), z) + Derivative(V_z(rho, phi, z), phi)/rho],
[                                   Derivative(V_\rho(rho, phi, z), z) - Derivative(V_z(rho, phi, z), rho)],
[Derivative(V_\phi(rho, phi, z), rho) + V_\phi(rho, phi, z)/rho - Derivative(V_\rho(rho, phi, z), phi)/rho]])

In [51]:
aa.Laplacian_scalar()

Derivative(F(rho, phi, z), (rho, 2)) + Derivative(F(rho, phi, z), (z, 2)) + Derivative(F(rho, phi, z), rho)/rho + Derivative(F(rho, phi, z), (phi, 2))/rho**2

In [52]:
aa.Laplacian_vector_full()

Matrix([
[Derivative(V_\rho(rho, phi, z), (rho, 2)) + Derivative(V_\rho(rho, phi, z), (z, 2)) + Derivative(V_\rho(rho, phi, z), rho)/rho - V_\rho(rho, phi, z)/rho**2 - 2*Derivative(V_\phi(rho, phi, z), phi)/rho**2 + Derivative(V_\rho(rho, phi, z), (phi, 2))/rho**2],
[Derivative(V_\phi(rho, phi, z), (rho, 2)) + Derivative(V_\phi(rho, phi, z), (z, 2)) + Derivative(V_\phi(rho, phi, z), rho)/rho - V_\phi(rho, phi, z)/rho**2 + Derivative(V_\phi(rho, phi, z), (phi, 2))/rho**2 + 2*Derivative(V_\rho(rho, phi, z), phi)/rho**2],
[                                                                                         Derivative(V_z(rho, phi, z), (rho, 2)) + Derivative(V_z(rho, phi, z), (z, 2)) + Derivative(V_z(rho, phi, z), rho)/rho + Derivative(V_z(rho, phi, z), (phi, 2))/rho**2]])

In [53]:
aa.Laplacian_vector_reduced()

Matrix([
[\nabla^{2}(V_\rho(rho, phi, z)) - V_\rho(rho, phi, z)/rho**2 - 2*Derivative(V_\phi(rho, phi, z), phi)/rho**2],
[\nabla^{2}(V_\phi(rho, phi, z)) - V_\phi(rho, phi, z)/rho**2 + 2*Derivative(V_\rho(rho, phi, z), phi)/rho**2],
[                                                                                \nabla^{2}(V_z(rho, phi, z))]])

### Example vector field to transform

In [54]:
# set up the original vector field
x, y, z = sy.symbols('x y z')
f, g, h = sy.symbols('f g h', cls=sy.Function)
f = 3*x
g = 4*y
h = 0
V = sy.Matrix((f, g, h))
V

Matrix([
[3*x],
[4*y],
[  0]])

In [55]:
vp, ll, orig = aa.vec_components_and_length(V)
ws.pp(vp)

<IPython.core.display.Math object>

In [56]:
ws.pp(ll)

<IPython.core.display.Math object>

In [57]:
ws.pp(orig)

<IPython.core.display.Math object>

# 3D Cartesian coordinates

In [58]:
x, y, z          = sy.symbols('x y z', real=True)
X, Y, Z          = sy.symbols('X, Y, Z', real=True)
orig_coords      = (x, y, z)
primed_coords    = (X, Y, Z)
orig_from_primed = (X, Y, Z)

In [59]:
# an example vector field
f, g, h = sy.symbols('f g h', cls=sy.Function)
f, g, h = [Y, -X, 0]
V1 = sy.Matrix((f, g, h))
f, g, h = [0, -X**2, 0]
V2 = sy.Matrix((f, g, h))

Matrix([
[    0],
[-X**2],
[    0]])

In [60]:
zz = ws.Operators(orig_coords, primed_coords, orig_from_primed)

In [61]:
zz.curl(V1) # curl should be -2 in Z direction: see https://en.wikipedia.org/wiki/Curl_(mathematics)

Matrix([
[ 0],
[ 0],
[-2]])

In [62]:
zz.curl(V2) # curl should be -2X in the Z direction

Matrix([
[   0],
[   0],
[-2*X]])

In [63]:
zz.metric

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [64]:
zz.Z

Matrix([
[\hat{\mathbf{e_x}}],
[\hat{\mathbf{e_y}}],
[\hat{\mathbf{e_z}}]])

In [65]:
zz.Zp

Matrix([
[\hat{\mathbf{e_X}}],
[\hat{\mathbf{e_Y}}],
[\hat{\mathbf{e_Z}}]])

In [66]:
zz.gradient(vector=False)

Matrix([[\hat{\mathbf{e_X}}*Derivative(F(X, Y, Z), X) + \hat{\mathbf{e_Y}}*Derivative(F(X, Y, Z), Y) + \hat{\mathbf{e_Z}}*Derivative(F(X, Y, Z), Z)]])

In [67]:
zz.gradient()

Matrix([
[Derivative(F(X, Y, Z), X)],
[Derivative(F(X, Y, Z), Y)],
[Derivative(F(X, Y, Z), Z)]])

In [68]:
zz.divergence()

Derivative(V_X(X, Y, Z), X) + Derivative(V_Y(X, Y, Z), Y) + Derivative(V_Z(X, Y, Z), Z)

In [69]:
zz.curl()

Matrix([
[-Derivative(V_Y(X, Y, Z), Z) + Derivative(V_Z(X, Y, Z), Y)],
[ Derivative(V_X(X, Y, Z), Z) - Derivative(V_Z(X, Y, Z), X)],
[-Derivative(V_X(X, Y, Z), Y) + Derivative(V_Y(X, Y, Z), X)]])

In [70]:
zz.Laplacian_scalar()

Derivative(F(X, Y, Z), (X, 2)) + Derivative(F(X, Y, Z), (Y, 2)) + Derivative(F(X, Y, Z), (Z, 2))

In [71]:
zz.Laplacian_vector_full()

Matrix([
[Derivative(V_X(X, Y, Z), (X, 2)) + Derivative(V_X(X, Y, Z), (Y, 2)) + Derivative(V_X(X, Y, Z), (Z, 2))],
[Derivative(V_Y(X, Y, Z), (X, 2)) + Derivative(V_Y(X, Y, Z), (Y, 2)) + Derivative(V_Y(X, Y, Z), (Z, 2))],
[Derivative(V_Z(X, Y, Z), (X, 2)) + Derivative(V_Z(X, Y, Z), (Y, 2)) + Derivative(V_Z(X, Y, Z), (Z, 2))]])

In [72]:
zz.Laplacian_vector_reduced()

Matrix([
[\nabla^{2}(V_X(X, Y, Z))],
[\nabla^{2}(V_Y(X, Y, Z))],
[\nabla^{2}(V_Z(X, Y, Z))]])